In [126]:
import numpy as np
import matplotlib.pyplot as plt

# Sample 2D time series data (rows represent time, columns represent features)
x = np.array([[1, 2, 3],
              [2, 3, 4]])
y = np.array([[5, 4, 3],
              [4, 3, 2],
              [3, 2, 1],
              [2, 1, 0]])

# a,b = zero_pad(x[:,0], y[:,0])
a,b = (x,y) #(x[:,0], y[:,0])

a_scale=-1+2*(a-np.min(np.concatenate((a,b)),axis=0)) / (np.max(np.concatenate((a,b)),axis=0)-np.min(np.concatenate((a,b)),axis=0))
b_scale=-1+2*(b-np.min(np.concatenate((a,b)),axis=0)) / (np.max(np.concatenate((a,b)),axis=0)-np.min(np.concatenate((a,b)),axis=0))

# Calculate the maximum possible lag
max_lag = len(y) - len(x) + 1

# Initialize array to store cross-correlation values for each lag
cross_corr_values = []


# Calculate cross-correlation for each possible alignment
# for lag in range(max_lag):
#     print('lag %d'%lag)
#     # Select the corresponding segment of y
#     y_segment = y[lag:lag + len(x)]
#     # Calculate cross-correlation between x and the selected segment of y
#     print(x[:,0])
#     print(y_segment[:,0])
#     cross_corr_value = np.correlate(x[:,0], y_segment[:,0], mode='valid')
#     cross_corr_values.append(cross_corr_value)
#     print(cross_corr_value[0])


In [127]:
def zero_pad(x,y):
    len_x = len(x)
    len_y = len(y)
    if len_x > len_y:
        y = np.pad(y, (0,len(x)-len(y)), 'constant', constant_values=(0))
    elif len_x < len_y:
        x = np.pad(x, (0,len(y)-len(x)), 'constant', constant_values=(0))
    return (x, y)

def sort_arrays_by_len(x,y):
    if len(x) >= len(y):
        return x, y
    else:
        return y, x
def sliding_diff(x,y):
    x, y = sort_arrays_by_len(x,y)
    max_lag = len(x)-len(y)+1
    # print(max_lag)
    len_y = len(y)
    total_diff = 0
    for lag in range(max_lag):
        # print('lag %d'%lag)
        diff = (x[lag:len_y+lag]-y)**2
        total_diff += diff.mean()
        # print(diff.mean())
    return total_diff
    



In [128]:
sliding_diff(a, b)

13.0

In [135]:
import pandas as pd
import json 

def str_list_to_array(str_list):
    return np.array([json.loads(v) for v in str_list])

track_info = pd.read_csv('./data/AudioFeats_MustBeThePlace.csv')
audio_df1 = pd.read_csv('./data/aud_analys_data_MustBePlace.csv')
audio_df1.head(3)

,track_name,track_id,start,duration,end,confidence,loudness_start,loudness_max_time,loudness_max,loudness_end,attack_aggression,pitches,timbre
0,This Must Be the Place (Naive Melody) - 2005 R...,6aBUnkXuCEQQHAlTokv9or,0.00000,0.13392,0.13392,0.0,-60.000,0.00000,-60.000,0.0,NaN,"[1.0, 0.977, 0.713, 0.71, 0.654, 0.626, 0.561,...","[0.011, 170.887, 9.19, -28.789, 57.303, -50.12..."
1,This Must Be the Place (Naive Melody) - 2005 R...,6aBUnkXuCEQQHAlTokv9or,0.13392,0.28408,0.41800,1.0,-60.000,0.03978,-10.570,0.0,1242.584213,"[0.266, 0.214, 0.459, 0.851, 1.0, 0.442, 0.382...","[33.232, 73.877, -196.871, 126.219, 17.257, 11..."
2,This Must Be the Place (Naive Melody) - 2005 R...,6aBUnkXuCEQQHAlTokv9or,0.41800,0.27814,0.69614,1.0,-42.018,0.01052,-6.792,0.0,3348.479087,"[0.26, 0.194, 0.253, 0.428, 0.67, 0.449, 0.498...","[39.723, 89.491, -175.686, 179.658, -26.575, -..."


In [209]:
# timbre_vals=str_list_to_array(audio_df1.timbre.tolist())
# timbre_df = pd.DataFrame(data=timbre_vals, columns=['timbre_%d'%i for i in range(timbre_vals.shape[1])])
scale_timbre=False
timbre_vals=[]
pitch_vals=[]
for track_id in track_info.track_id.tolist():
    timbre = audio_df1[audio_df1.track_id==track_id].timbre.values.tolist()
    timbre = str_list_to_array(timbre)

    pitch = audio_df1[audio_df1.track_id==track_id].pitches.values.tolist()
    pitch = str_list_to_array(pitch)
    
    timbre_vals += [timbre]
    pitch_vals += [pitch]

In [210]:
X = [ np.concatenate((t,p),axis=1) for t,p in zip(timbre_vals,pitch_vals)]
X[0]

array([[ 1.10000e-02,  1.70887e+02,  9.19000e+00, ...,  5.06000e-01,
         4.47000e-01,  9.00000e-02],
       [ 3.32320e+01,  7.38770e+01, -1.96871e+02, ...,  2.69000e-01,
         8.20000e-02,  2.66000e-01],
       [ 3.97230e+01,  8.94910e+01, -1.75686e+02, ...,  6.10000e-01,
         1.00000e+00,  4.54000e-01],
       ...,
       [ 6.13500e+00,  8.23540e+01, -1.17000e-01, ...,  4.06000e-01,
         6.59000e-01,  1.74000e-01],
       [ 6.61100e+00,  9.07550e+01, -2.81150e+01, ...,  3.26000e-01,
         5.42000e-01,  7.97000e-01],
       [ 3.37600e+00,  1.17380e+02, -4.03940e+01, ...,  1.00000e+00,
         7.31000e-01,  6.32000e-01]])

In [211]:
X_max = np.max(np.concatenate(X, axis=0), axis=0)
X_min = np.min(np.concatenate(X, axis=0), axis=0)


array([[1.   , 0.977, 0.713, ..., 0.506, 0.447, 0.09 ],
       [0.266, 0.214, 0.459, ..., 0.269, 0.082, 0.266],
       [0.26 , 0.194, 0.253, ..., 0.61 , 1.   , 0.454],
       ...,
       [0.176, 0.148, 0.102, ..., 1.   , 0.247, 0.119],
       [0.04 , 0.023, 0.042, ..., 1.   , 0.164, 0.056],
       [0.17 , 0.157, 0.086, ..., 1.   , 0.174, 0.113]])

In [201]:
D = np.ones((len(X),len(X))) * np.inf

for i in range(len(X)):
    x = X[i]
    x = -1+2*(x-X_min)/(X_max-X_min)
    for j in range(i+1,len(X)):
        y = X[j]
        y = -1+2*(y-X_min)/(X_max-X_min)
        D[i,j] = sliding_diff(x,y)

In [202]:
np.argsort(D, axis=1)

array([[14, 28, 74, ..., 38, 49,  0],
       [64, 15, 72, ..., 49,  1,  0],
       [44, 53, 34, ...,  2,  1,  0],
       ...,
       [75, 76, 73, ..., 20, 27, 38],
       [76, 54, 53, ..., 20, 27, 38],
       [ 0, 54, 53, ..., 20, 27, 76]])

In [203]:
np.argmax(np.nan_to_num(D, posinf=0), axis=1)


array([49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49,
       49, 49, 49, 49, 49, 49, 49, 45, 49, 49, 49, 49, 49, 49, 49, 49, 49,
       49, 49, 49, 49, 45, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 63, 56,
       63, 59, 59, 59, 59, 59, 59, 73, 63, 73, 63, 63, 65, 73, 66, 73, 73,
       73, 71, 73, 73, 73, 75, 76, 76,  0])

In [204]:
ix_inspect = [np.unravel_index(ix, D.shape) for ix in np.argsort(D.flatten())[:5]]
print(ix_inspect)
print(np.sort(D.flatten())[:5])

[(21, 29), (50, 65), (3, 35), (60, 68), (18, 56)]
[0.36908112 0.37127311 0.38320532 0.38852455 0.70142808]


In [206]:
track_info.iloc[[3,35],:]

,track_name,track_id,duration_ms,time_signature,tempo,valence,liveness,instrumentalness,acousticness,speechiness,mode,loudness,key,energy,danceability,loudness_scaled
3,Tieduprightnow,66tkDkPsznE5zIHNt4QkXB,249957,4,102.025,0.738,0.0794,0.001260,0.2910,0.0484,0,-7.986,5,0.533,0.863,0.866900
35,The Passenger,15BQ7vEDv2LJuh8TxWIhtd,283360,4,134.880,0.696,0.7330,0.000002,0.0947,0.0546,1,-5.413,0,0.889,0.498,0.909783
